In [1]:
import pandas as pd
import datetime
import numpy as np
import re
from re import search

In [2]:
# replace values and create new Log_jeden_den2.txt file

with open("log.txt", 'r') as infile, open('Log_jeden_den2.txt', 'w') as outfile:
    logFile = infile.read()
    logFile = logFile.replace("-", "")
    logFile = logFile.replace(" +0100", "+0100")
    logFile = logFile.replace(" /", "/")
    logFile = logFile.replace(" HTTP", "HTTP")
    logFile = logFile.replace("   ", " ")
    outfile.write(logFile)

In [3]:
# ResponseCode = navratovy kod ... napriklad 404-not found..
# Refferer = zaznamenava, ze odkial som prisiel na danú stranku..takze je to "predchadzajuca stranka"
# UserAgent = cez aky prehliadac pristupoval User na stranku, informacie o OS atd.

colNames = ["IP", "DateTime", "URL", "ResponseCode","Bytes","Referrer","UserAgent","UserId","UNIXTIME"]
df = pd.read_csv("Log_jeden_den2.txt", sep=" ", names=colNames)

C:\Users\micha\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4,5,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
# replace values NaN with "-" in column referrer

df['Referrer'].fillna("-",inplace=True)

In [5]:
# delete rows with ResponseCodes (1xx, 4xx, 5xx) 

df["ResponseCode"]= df["ResponseCode"].apply(str) 
deleteRowsWithValues = ["1", "4", "5"]
df = df[~df["ResponseCode"].str[:1].str.contains('|'.join(deleteRowsWithValues))]

In [6]:
# delete rows with URL ["POST", "HEAD" ...] 
deleteRowsWithValues = ["POST", "HEAD",".bmp", ".jpg", ".jpeg", ".png", ".gif", ".JPG", ".css", ".flv", ".ico", ".swf"
                       , ".rss", ".xml", ".cur", ".js", ".json", ".svg", ".woff", ".eot"]
df = df[~df["URL"].str.contains('|'.join(deleteRowsWithValues))]

In [7]:
for x in df['URL']:
        if re.search("navbar",x) or re.search("cron",x):
            df.drop(df.index[df['URL'] == x], inplace = True)

In [8]:
# set same ID for the same IPs
IDnum = df.IP.unique()                          
IDnum = pd.Series(np.arange(len(IDnum)), IDnum) 
df["UserId"] = df.applymap(IDnum.get)

In [9]:
# vytvorenie z Datetime na unixtime pomocou for cyklu  zistenia pozície danej hodnoty 
i=0
for x in df['DateTime']:
    if len(str(x))>21:
        df.iloc[i, 8]=datetime.datetime(2020,2,int(x[1:3]),int(x[14]),int(x[16:18]),int(x[19:21])).timestamp()
        i=i+1
    else:
        df.iloc[i, 8]='-'
        i=i+1

In [10]:
data=df[df['URL'].str.contains("robots.txt")]
z=pd.unique(data['IP'])
len(z)

0

In [11]:
for x in data['IP']:
    for y in z:
        if x==y:
            df.drop(df.index[df['IP'] == x], inplace = True)

In [12]:
df['UserAgent']=df['UserAgent'].apply(str)
for x in df['UserAgent']:
    if re.search("bot",x) or re.search("crawl",x) or re.search("spider",x):
        df.drop(df.index[df['UserAgent'] == x], inplace = True)

In [13]:
len(df)

1724418

In [14]:
df2=df.sort_values(['IP','UserAgent','UNIXTIME'], ascending=True)
df2["id"] =""
df2['id'] = df2.groupby(['IP','UserAgent']).ngroup()


In [15]:
df2.head(7)

,IP,DateTime,URL,ResponseCode,Bytes,Referrer,UserAgent,UserId,UNIXTIME,id
1084070,1.10.198.104,[20/Sep/2017:18:28:49,+0200],GET/wplogin.phpHTTP/1.1,302,3778,nan,13735,-,0
1084071,1.10.198.104,[20/Sep/2017:18:28:50,+0200],GET/index.php?option=com_content&view=article&...,200,38691,nan,13735,-,0
1084073,1.10.198.104,[20/Sep/2017:18:28:51,+0200],GET/HTTP/1.1,200,50184,nan,13735,-,0
6106,1.10.213.183,[17/Sep/2017:08:56:54,+0200],GET/wplogin.phpHTTP/1.1,302,3245,nan,324,-,1
6107,1.10.213.183,[17/Sep/2017:08:56:55,+0200],GET/index.php?option=com_content&view=article&...,200,39411,nan,324,-,1
6108,1.10.213.183,[17/Sep/2017:08:56:57,+0200],GET/HTTP/1.1,200,51592,nan,324,-,1
175837,1.121.136.130,[17/Sep/2017:02:32:54,+0200],GET/wplogin.phpHTTP/1.1,302,3778,nan,3966,-,2


In [16]:
df2.to_csv(r'simko2.csv',  index=None)

In [17]:
df3 = pd.read_excel("simko3.xlsx")

In [20]:
SequenceID = 3600 # je čas sekunach (zadaný v zadaní)
length = []
index = 0
nextIndex = 1 #cyklus ide od 1 po dataframe 
for i in range(1,len(df3)):
    if((df2.iloc[nextIndex,[7]].values == df2.iloc[index,[7]].values) and ((df2.iloc[nextIndex,[8]].values-df2.iloc[index,[8]].values) < SequenceID)):
        length.append(str(df2.iloc[nextIndex,[8]].values-df2.iloc[index,[8]].values))
        index+=1
        nextIndex+=1
    else:
        length.append("")
        index+=1
        nextIndex+=1

length.append("") # priradí hodnotu stplci

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [ ]:
df3["length"] = length

In [ ]:
df3["length"] = df3["length"].apply(lambda x: x.replace("[", "")) # odstranime znaky 
df3["length"] = df3["length"].apply(lambda x: x.replace("]", ""))

In [ ]:
df3['length'] = pd.to_numeric(df3['length']) #zmenime hodnotu length na čiselnu hodnotu

In [ ]:
df3.to_csv(r'simko3.csv',  index=None)

In [ ]:
df4 = pd.read_csv("simko3.csv")

In [ ]:
# C = -ln(1-p)/ lambda ... kde lambda = 1/rlength...
#lambda = 1/priemerna intenzita udalosti alebo priemerna dlzka navstev

podielNavigacnychStranok = 40 # % .. hodnota bola uvedena v ulohe
p = podielNavigacnychStranok/100 # p je relativna pocetnost navigacnych stranok, 0 <= p < 1 ... v nasom pripade to bude 40/100 = 0,40
hodnotaLambda = df4["length"].mean() # priemerna dlzka navstev alebo priemerna intenzia udalosti
c = -np.log(1-p)/(1/hodnotaLambda)

In [ ]:
c 

In [ ]:
df4.info()

In [ ]:
sedenia = []
sedeniaNumber = 1 # zaciname od jednotky
i = 0
pamat = 0  
while i < 20567:
    if(df4.iloc[i,7] == df.iloc[i+1,7]): # ked userID na indexe i sa bude rovnat nasledujucemu riadku
        pamat += df4.iloc[i,10] # v pamati ulozime riadok i zo stlpca length
        if(pamat < c): # ked hodnota premennej pamat je mensia ako c (hranicny cas)
            sedenia.append(sedeniaNumber) # pridavame do zoznamu
        else:
            sedenia.append(sedeniaNumber) 
            pamat = 0 #kedze to je uz cast else, musime hodnotu pamati vynulovat
            sedeniaNumber+=1 # zvysime hodnotu sedeniaNumber o 1
        i+=1 
    else:
        pamat = 0
        sedenia.append(sedeniaNumber)
        sedeniaNumber+=1
        i+=1
sedenia.append(sedeniaNumber)

In [ ]:
df4["RLength"] = sedenia # pridavame novy stlpec s hodnotami sedenia do nasho dataframu

In [ ]:
df4.to_csv(r'simko4.csv',  index=None)